In [ ]:
%pip install anywidget

In [ ]:
import anywidget
import traitlets
from ipywidgets import FloatSlider

In [ ]:
## Create the humidity widget class, which communicates with the Phidget devices.

class humidity_widget(anywidget.AnyWidget):
    _esm = """

import {USBConnection, HumiditySensor} from "https://esm.sh/phidget22@3.17";

export function render({ model, el }) {
    var conn = 0; // the USB connection
    var chan = 0; // the channel for the device
    var isOpen = false;
    var connOpen = false;
    var chanOpen = false;

    async function openUSB() {
        if (conn) {
            try {await chan.close();} catch {}
            try {await conn.close();} catch {}
            try {await conn.delete();} catch {}
        }
        conn = new USBConnection();
        chan = new HumiditySensor();
        chan.onHumidityChange = function(value) {
            textValue.innerHTML = 'Humidity is ' + value;
            model.set('value', value);
            model.save_changes();
       	};
        chan.onAttach = async () => {
            textStatus.innerHTML = 'Attached. ';
            await chan.setDataInterval(250);
        }

        connOpen = true;
        try {console.log('conn connect'); await conn.connect();} 
            catch(err) {console.log('connect error ' + err); connOpen = false;}
        try {console.log('conn request'); await conn.requestWebUSBDeviceAccess();} 
            catch(err) {console.log('request error ' + err); connOpen = false;}
        if (connOpen) {
            chanOpen = false;
            try {
                await chan.open(5000);
                chanOpen = true;
            } catch(err) {chanOpen = false;}
        }
        
        // now let the UI reflect the status of the connections
        if (connOpen && chanOpen) {
            isOpen = true;
            textStatus.innerHTML = 'USB connected, channel open. ';
            button.innerHTML = `Click to disconnect`;
        }
        if (connOpen && !chanOpen) {
            isOpen = false;
            textStatus.innerHTML = 'USB connected, channel not open. ';
            button.innerHTML = `Click to connect`;
        }
        if (!connOpen) {
            isOpen = false;
            textStatus.innerHTML = 'USB did not connect. ';
            button.innerHTML = `Click to connect`;        
        }
    };
    async function closeUSB() {
        try {await chan.close();} catch {}
        try {await conn.close();} catch {}
        try {await conn.delete();} catch {}
        isOpen = false; 
        connOpen = false;
        chanOpen = false;
        textStatus.innerHTML = 'Disconnected. ';
        textValue.innerHTML = 'Humidity is null. ';
        button.innerHTML = `Click to connect`;
    };
    
    // here we define the user interface, a button and two text boxes
    let button = document.createElement("button");
    button.classList.add("ph-button");
    button.innerHTML = `Click to open USB`;
    button.addEventListener("click", async () => {
        if (isOpen) {closeUSB();} else {openUSB();}
    });
    let textStatus = document.createElement("label");
    textStatus.innerHTML = 'Status message here. ';
    let textValue = document.createElement("label");
    textValue.innerHTML = 'Humidity is null. ';
    
    // Post the UI into the Jupyter notebook cell
    el.appendChild(button);
    el.appendChild(textStatus);
    el.appendChild(textValue);
    
    // we include a return function to close the Phidget when the notebook is closed
    return closeUSB();
  }
    """
    _css = """
    .ph-button {color: white; 
        background-color:rgb(96, 107, 174); 
        border-radius: 8px; 
        font-size: 24px; 
        display: block;
        padding: 15px 32px;}
    .ph-button:hover { background-color:rgb(120, 128, 187); }  
    """
    value = traitlets.Float(0).tag(sync=True)

In [ ]:
## Create the humidity widget and link it to a slider, representing the humidity level.

phidget = humidity_widget()
gauge = FloatSlider(min=20,max=80.0,orientation='horizontal',description = 'Humidity')
def updateGauge(change):
    gauge.value = change.new

phidget.observe(updateGauge, names=['value'])

In [ ]:
## Display the humidity widget and the slider. 

display(phidget,gauge)